In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show(truncate=False)

+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')
df.printSchema()


root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:
# Changing Datatypes
df = df.withColumn("date", df["date"].cast('timestamp')) \
        .withColumn("date_built", df["date_built"].cast('float')) \
        .withColumn("price", df["price"].cast('float')) \
        .withColumn("bathrooms", df["bathrooms"].cast('float')) \
        .withColumn("bedrooms", df["bedrooms"].cast('float')) \
        .withColumn("sqft_living", df["sqft_living"].cast('float')) \
        .withColumn("sqft_lot", df["sqft_lot"].cast('float')) \
        .withColumn("floors", df["floors"].cast('float')) \
        .withColumn("waterfront", df["waterfront"].cast('float')) \
        .withColumn("view", df["view"].cast('float'))
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- date_built: float (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: float (nullable = true)
 |-- bathrooms: float (nullable = true)
 |-- sqft_living: float (nullable = true)
 |-- sqft_lot: float (nullable = true)
 |-- floors: float (nullable = true)
 |-- waterfront: float (nullable = true)
 |-- view: float (nullable = true)



In [7]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
        SELECT YEAR(date) AS year_sold, ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        GROUP BY year_sold
        ORDER BY year_sold DESC
        """).show()


+---------+-------------+
|year_sold|average_price|
+---------+-------------+
|     2022|    314421.54|
|     2021|    313218.09|
|     2020|    315949.81|
|     2019|    315782.18|
+---------+-------------+



In [8]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
        SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        WHERE bedrooms = 3 AND bathrooms = 3
        GROUP BY year_built
        ORDER BY year_built DESC
        """).show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2017|    292676.79|
|      2016|    290555.07|
|      2015|     288770.3|
|      2014|    290852.27|
|      2013|    295962.27|
|      2012|    293683.19|
|      2011|    291117.47|
|      2010|    292859.62|
+----------+-------------+



In [9]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
        SELECT date_built AS year_built, ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
        GROUP BY year_built
        ORDER BY year_built DESC
        """).show()


+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [10]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""
        SELECT view AS view_rating, ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        WHERE price >= 350000
        GROUP BY view_rating
        ORDER BY view_rating DESC
        """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|          9|    401393.34|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 1.208132266998291 seconds ---


In [11]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [12]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [13]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql("""
        SELECT view AS view_rating, ROUND(AVG(price), 2) AS average_price
        FROM home_sales
        WHERE price >= 350000
        GROUP BY view_rating
        ORDER BY view_rating DESC
        """).show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|         99|   1061201.42|
|         98|   1053739.33|
|         97|   1129040.15|
|         96|   1017815.92|
|         95|    1054325.6|
|         94|    1033536.2|
|         93|   1026006.06|
|         92|    970402.55|
|         91|   1137372.73|
|         90|   1062654.16|
|          9|    401393.34|
|         89|   1107839.15|
|         88|   1031719.35|
|         87|    1072285.2|
|         86|   1070444.25|
|         85|   1056336.74|
|         84|   1117233.13|
|         83|   1033965.93|
|         82|    1063498.0|
|         81|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 0.6341524124145508 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [15]:
# 11. Read the formatted parquet data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [16]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_p')

In [17]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""
        SELECT view AS view_rating, ROUND(AVG(price), 2) AS average_price
        FROM p_home_sales_p
        WHERE price >= 350000
        GROUP BY view_rating
        ORDER BY view_rating DESC
        """).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------+-------------+
|view_rating|average_price|
+-----------+-------------+
|      100.0|    1026669.5|
|       99.0|   1061201.42|
|       98.0|   1053739.33|
|       97.0|   1129040.15|
|       96.0|   1017815.92|
|       95.0|    1054325.6|
|       94.0|    1033536.2|
|       93.0|   1026006.06|
|       92.0|    970402.55|
|       91.0|   1137372.73|
|       90.0|   1062654.16|
|       89.0|   1107839.15|
|       88.0|   1031719.35|
|       87.0|    1072285.2|
|       86.0|   1070444.25|
|       85.0|   1056336.74|
|       84.0|   1117233.13|
|       83.0|   1033965.93|
|       82.0|    1063498.0|
|       81.0|   1053472.79|
+-----------+-------------+
only showing top 20 rows

--- 1.5399560928344727 seconds ---


In [18]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [19]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False